Look to refactor into the following model:

- between attacks, we have an unchanging state
- from that state, the next attack is chosen (randomly)
- compute the next state based on which attack was chosen

In [1]:
from dataclasses import dataclass
from typing import List, Tuple, Optional, Callable, Any
from itertools import cycle
import random
import json
from abc import ABC, abstractmethod
from collections import namedtuple
import json

In [2]:
NUM_POSITIONS = 7

In [37]:
Bonus = namedtuple("Bonus", ["attack", "health"])

in_front_of = {
    0: [],
    1: [],
    2: [],
    3: [],
    4: [0, 1],
    5: [1, 2],
    6: [2, 3]
}

In [38]:
@dataclass
class MinionDetails:
    """Minion characteristics and stats"""
    name: str
    base_attack: int
    base_health: int
    alignment: str
    types: List[str]
    level: int
    upgraded: bool = False
    ranged: bool = False
    flying: bool = False
    slay: bool = False

In [39]:
""" subscribers = dict()

def subscribe(event_type: str, fn):
    if not event_type in subscribers:
        subscribers[event_type] = []
    subscribers[event_type].append(fn)

def post_event(event_type: str, *args, **kwargs):
    if not event_type in subscribers:
        return
    for fn in subscribers[event_type]:
        fn(*args, **kwargs) """

' subscribers = dict()\n\ndef subscribe(event_type: str, fn):\n    if not event_type in subscribers:\n        subscribers[event_type] = []\n    subscribers[event_type].append(fn)\n\ndef post_event(event_type: str, *args, **kwargs):\n    if not event_type in subscribers:\n        return\n    for fn in subscribers[event_type]:\n        fn(*args, **kwargs) '

In [91]:
class Minion:

    def __init__(self, details: MinionDetails, position: int, mediator: "MinionMediator"):
        self.attack = details.base_attack
        self.health = details.base_health
        self.details = details
        self.position = position
        self.mediator = mediator
    
    def support(self) -> Tuple[Bonus, Callable[["Minion"], bool]]:
        return (0, 0), lambda x: False 

    def apply_bonus(self, bonus: Bonus) -> None:
        self.attack += bonus.attack
        self.health += bonus.health

    def take_damage(self, damage: int) -> None:
        self.health -= damage
        if self.health < 0:
            self.mediator.kill_minion(self.position)

    def on_death(self) -> Callable:
        return lambda: None

    @property
    def can_attack(self) -> bool:
        return self.attack > 0

    def __repr__(self) -> str:
        return f"{self.details.name} in position {self.position} with {self.attack} attack and {self.health} health"

In [92]:
minion_classes = {}
def register(name):
    def wrapper(cls):
        minion_classes[name] = cls
        return cls
    return wrapper

@register(name='Mad Mim')
class MadMim(Minion):

    def support(self) -> Tuple[Bonus, Callable[[Minion], bool]]:
        return Bonus(3, 0), lambda x: x.position in in_front_of[self.position]

@register(name='Rainbow Unicorn')
class RainbowUnicorn(Minion):

    def support(self) -> Tuple[Bonus, Callable[[Minion], bool]]:
        return Bonus(0, 1), lambda x: x.details.alignment == 'Good' and x is not self

@register(name='Black Cat')
class BlackCat(Minion):

    def on_death(self) -> None:
        return lambda: self.mediator.add_minion('Cat', details=MinionDetails(**ALL_MINIONS['Cat']), position=self.position)

@register(name='Cat')
class Cat(Minion):
    pass

@register(name='Happy Little Tree')
class HappyLittleTree(Minion):
    pass

In [83]:
minion_classes

{'Mad Mim': __main__.MadMim,
 'Rainbow Unicorn': __main__.RainbowUnicorn,
 'Black Cat': __main__.BlackCat,
 'Cat': __main__.Cat,
 'Happy Little Tree': __main__.HappyLittleTree}

In [102]:
@dataclass
class AttackDetails:
    attack: int
    flying: bool = False
    ranged: bool = False

FRONT_ROW = (0, 1, 2, 3)
BACK_ROW = (4, 5, 6)

In [119]:
class MinionMediator:

    def __init__(self, minions: List[Tuple[MinionDetails, int]]):
        self.minions = {}
        for details, position in minions:
            self.add_minion(details.name, details, position)
        self.last_attacker = None
        self.apply_supports()

    def add_minion(self, name: str, details: MinionDetails, position: int) -> None:
        if position not in self.minions:
            self.minions[position] = minion_classes[name](details=details, position=position, mediator=self)

    def remove_minion(self, position: int) -> None:
        del self.minions[position]

    def kill_minion(self, position: int) -> None:
        on_death = self.minions[position].on_death()
        self.remove_minion(position)
        on_death() # This will later be placed on a stack

    def receive_attack(self, attack_details: AttackDetails) -> int:
        """Accept incoming attack, and return the damage to be dealt back"""
        assert self.has_minions
        defender = self._get_defender(attack_details)
        defender.take_damage(attack_details.attack)
        return defender.attack if attack_details.ranged else 0

    def _get_defender(self, attack: AttackDetails) -> Minion:
        front = [minion for minion in self.minions.values() if minion.position in FRONT_ROW]
        back = [minion for minion in self.minions.values() if minion.position in BACK_ROW]
        if len(front) == 0:
            return random.choice(back)
        elif len(back) == 0 or not attack.flying:
            return random.choice(front)
        else: # attack.flying
            return random.choice(back)
        
    def _attack_priority(self, minion: Minion) -> int:
        """Integer value to determine attacking order, lowest value has highest priority"""
        if minion is self.last_attacker:
            return NUM_POSITIONS * 3 # ensure lowest priority 
        elif minion.position >= (self.last_attacker.position if self.last_attacker else 0):
            return minion.position
        else:
            return minion.position + NUM_POSITIONS 

    @property
    def has_attacker(self) -> bool:
        return any(m.can_attack for m in self.minions.values())

    def _get_attacker(self) -> Minion:
        for minion in sorted(self.minions.values(), key=self._attack_priority):
            if minion.can_attack:
                return minion
        else:
            raise RuntimeError("Tried to get attacker but none available.")

    def attack(self, opponent: MinionMediator) -> None:
        if not self.has_attacker:
            return
        attacker = self._get_attacker()
        attack_details = AttackDetails(attack=attacker.attack, ranged=attacker.details.ranged, flying=attacker.details.flying)
        damage_taken = opponent.receive_attack(attack_details)
        attacker.take_damage(damage_taken)
        self.last_attacker = attacker

    def apply_supports(self) -> None:
        for supporter in self.minions.values():
            bonus, criteria = supporter.support()
            for target in filter(criteria, self.minions.values()):
                target.apply_bonus(bonus)

    @property
    def has_minions(self) -> bool:
        return len(self.minions) > 0

In [131]:
class Game:

    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2
        self.turn_sequence = cycle([(self.p1, self.p2), (self.p2, self.p1)]) 
        if random.choice([True, False]):
            next(self.turn_sequence)

    def run(self) -> float:
        for attacker, opponent in self.turn_sequence:
            if self.is_terminal():
                return self.game_result()
            attacker.attack(opponent)

    def is_terminal(self) -> bool:
        return (not self.p1.has_minions) or (not self.p2.has_minions) or (not self.p1.has_attacker and not self.p2.has_attacker)

    def game_result(self) -> float:
        if (not self.p1.has_minions) and (not self.p2.has_minions):
            return 0.5 # draw
        elif not self.p1.has_minions:
            return 0
        elif not self.p2.has_minions:
            return 1
        elif not self.p1.has_attacker and not self.p2.has_attacker:
            return 0.5 # draw
        else:
            raise RuntimeError("Game hasn't ended!")

In [140]:
with open('minions.json') as f:
    ALL_MINIONS = json.loads(f.read())

def play():
    p1 = MinionMediator([
        (MinionDetails(**ALL_MINIONS['Mad Mim']), 4), 
        (MinionDetails(**ALL_MINIONS['Rainbow Unicorn']), 0), 
        (MinionDetails(**ALL_MINIONS['Black Cat']), 1),
        (MinionDetails(**ALL_MINIONS['Happy Little Tree']), 3)
    ])
    p2 = MinionMediator([
            (MinionDetails(**ALL_MINIONS['Mad Mim']), 4), 
            (MinionDetails(**ALL_MINIONS['Rainbow Unicorn']), 0), 
            (MinionDetails(**ALL_MINIONS['Black Cat']), 1),
            (MinionDetails(**ALL_MINIONS['Happy Little Tree']), 3)
        ])
    game = Game(p1, p2)
    return game.run()

In [141]:
TRIALS = 1000
sum([play() for _ in range(TRIALS)]) / TRIALS


0.499

In [15]:
player = Player(('Baby Dragon', None, 'Black Cat', None, 'Mad Mim', 'Baby Root', None))
player.resolve_supports()
assert player.positions[0].attack == 6
assert player.positions[0].health == 2
assert player.positions[2].attack == 1
assert player.positions[2].health == 4

In [13]:
# test next_position

player = Player(('Mad Mim', None, None, None, 'Baby Dragon', None, None))
assert player.next_position() == 4
player.current_position = 4
player.last_uid = id(player.positions[4])
assert player.next_position() == 4

player = Player(('Mad Mim', None, None, None, None, None, None))
assert player.next_position() is None

player = Player(('Baby Dragon', None, None, None, 'Black Cat', None, None))
assert player.next_position() == 0
player.current_position = 0
player.last_uid = id(player.positions[0])
assert player.next_position() == 4
player.current_position = 4
player.last_uid = id(player.positions[4])
assert player.next_position() == 0